In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
from PIL import Image
import glob

/Users/hanjun/opt/anaconda3/envs/pt38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
img_list = glob.glob('/Users/hanjun/Desktop/python/비타민/추천시스템/중복처리완료_데이터/*')
len(img_list)

1117

In [3]:
model = models.resnet18(pretrained=True)

feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])

x = torch.randn([1,3,224,224])

output = feature_extractor(x)

print(output.shape)

/Users/hanjun/opt/anaconda3/envs/pt38/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/hanjun/opt/anaconda3/envs/pt38/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([1, 512, 1, 1])


In [11]:
feature_extractor

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [5]:
from tqdm import tqdm
import numpy as np
import os
import cv2
import pandas as pd

In [6]:
img_l = [x.split('/')[-1] for x in img_list]

In [7]:
img_l

['gyeongnam_203.jpg',
 'gyeongbuk_194.jpg',
 'gyeongnam_217.jpg',
 'jeju_102.jpg',
 'jeju_116.jpg',
 'gyeongbuk_143.jpg',
 'gyeongbuk_157.jpg',
 'gyeongbuk_24.jpg',
 'gyeongnam_9.jpg',
 'gyeongbuk_0.jpg',
 'gyeongbuk_18.jpg',
 'jeonnam_248.jpg',
 'jeju_248.jpg',
 'gyeongnam_175.jpg',
 'gyeongnam_161.jpg',
 'jeju_274.jpg',
 'gangwon_33.jpg',
 'gangwon_27.jpg',
 'gyeongbuk_235.jpg',
 'gangwon_165.jpg',
 'gangwon_171.jpg',
 'gyeongbuk_221.jpg',
 'gyeongnam_26.jpg',
 'gyeongbuk_209.jpg',
 'gangwon_159.jpg',
 'jeju_7.jpg',
 'gangwon_5.jpg',
 'jeju_63.jpg',
 'jeju_77.jpg',
 'jeonnam_67.jpg',
 'jeonnam_116.jpg',
 'jeonnam_102.jpg',
 'jeju_88.jpg',
 'jeju_89.jpg',
 'jeonnam_99.jpg',
 'jeonnam_103.jpg',
 'jeju_76.jpg',
 'jeonnam_66.jpg',
 'gangwon_4.jpg',
 'jeonnam_72.jpg',
 'gyeongbuk_208.jpg',
 'jeju_6.jpg',
 'gyeongbuk_220.jpg',
 'gyeongbuk_234.jpg',
 'gyeongnam_33.jpg',
 'gangwon_164.jpg',
 'jeju_275.jpg',
 'gyeongnam_148.jpg',
 'gangwon_32.jpg',
 'gyeongnam_160.jpg',
 'jeju_249.jpg',
 'gye

In [17]:
features_df = pd.DataFrame(img_l, columns=['img'])
features_df.head()

,img
0,gyeongnam_203.jpg
1,gyeongbuk_180.jpg
2,gyeongbuk_194.jpg
3,gyeongnam_217.jpg
4,jeju_102.jpg


In [27]:
model = models.resnet18(pretrained=True)

#feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
# feature_extractor_l3 = torch.nn.Sequential(*list(model.children())[:-3], list(model.children())[-2])
feature_extractor_l2 = torch.nn.Sequential(*list(model.children())[:-4], list(model.children())[-2])

device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Transform the image, so it becomes readable with the model
transform = transforms.Compose([
  transforms.ToPILImage(),
  transforms.Resize((224,224)),
  transforms.ToTensor()                              
])

# Will contain the feature
features = []
img_df = pd.DataFrame(columns=['img_name', 'feature'])

# Iterate each image
for i in tqdm(img_list):
  # Set the image path
  # Read the file
  img = cv2.imread(str(i))
	# Transform the image
  img = transform(img)
	# Reshape the image. PyTorch model reads 4-dimensional tensor
	# [batch_size, channels, width, height]
  img = img.view(1, 3, 224, 224)
  img = img.to(device)
	# We only extract features, so we don't need gradient
  with torch.no_grad():
		# Extract the feature from the image
    feature = feature_extractor_l2(img)
	# Convert to NumPy Array, Reshape it, and save it to features variable
  features.append(feature.cpu().detach().numpy().reshape(-1))
  # img_df = img_df.append({'img_name': str(i.split('/')[-1]), 'feature': feature.cpu().detach().numpy().reshape(-1).tolist()}, ignore_index=True)
# Convert to NumPy Array
features = np.array(features)

feature_df = pd.DataFrame(features)
img_l = pd.DataFrame(img_l, columns=['img_name'])

features_df = pd.concat([img_l, feature_df], axis=1)
features_df.to_csv('/Users/hanjun/Desktop/python/비타민/추천시스템/1117img_layer2_feature.csv')

/Users/hanjun/opt/anaconda3/envs/pt38/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Users/hanjun/opt/anaconda3/envs/pt38/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 1117/1117 [01:03<00:00, 17.61it/s]


In [17]:
features_df.shape

(1117, 257)

In [25]:
feature_df = pd.DataFrame(features)
img_l = pd.DataFrame(img_l, columns=['img_name'])

features_df = pd.concat([img_l, feature_df], axis=1)
features_df

,img_name,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,gyeongnam_203.jpg,0.859666,0.991506,0.875786,1.105204,0.882326,0.932529,0.958247,1.049270,0.957716,...,0.868787,0.939986,0.930962,0.891541,1.075375,0.981445,0.939782,0.905530,1.049204,0.956306
1,gyeongbuk_180.jpg,0.939454,1.032504,0.965032,1.120938,0.936345,0.855949,0.912005,1.178156,0.968077,...,0.868076,0.978992,0.898055,0.852248,1.145555,0.995994,0.941523,0.945611,1.087896,0.990869
2,gyeongbuk_194.jpg,0.887674,0.938588,0.984997,0.993302,0.952346,0.867004,0.949086,1.145820,0.954059,...,0.933420,0.918735,0.873764,0.814997,1.142438,0.961115,0.921529,0.869837,1.010402,0.949081
3,gyeongnam_217.jpg,0.865426,0.856954,0.942237,0.992056,0.896226,0.812261,0.973277,1.169686,0.909709,...,0.925228,0.860799,0.903770,0.865802,1.130654,1.060827,0.842281,0.886043,1.154529,0.931309
4,jeju_102.jpg,0.887677,0.977672,0.993547,1.024758,0.981420,0.837704,0.930538,1.237387,0.931367,...,0.877792,0.944481,0.853980,0.896813,1.073029,0.988909,0.937262,0.970793,0.991031,0.978287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1230,gyeongnam_232.jpg,0.930864,0.958179,0.954716,1.009896,0.896230,0.919095,0.903566,1.164350,0.987083,...,0.938534,0.903383,0.984284,0.834925,1.097964,0.901745,0.938423,0.987846,1.090714,0.983477
1231,gyeongnam_226.jpg,0.941443,0.945894,0.934900,1.055538,0.997482,0.881454,0.944459,1.271220,0.901844,...,0.959296,0.940265,0.899484,0.815066,1.180603,1.067878,0.888551,0.943487,1.095905,0.946472
1232,jeju_133.jpg,0.898361,1.030736,1.010446,1.037791,0.970882,0.858237,0.912649,1.244254,1.026163,...,0.887611,0.955979,0.854755,0.821449,1.224061,0.978837,0.976867,0.932357,1.063367,0.945248
1233,jeju_127.jpg,0.870150,0.951886,0.970361,1.045178,0.915710,0.932503,0.902959,1.152438,0.989491,...,0.919948,0.945434,0.854771,0.854165,1.087450,1.016544,1.010237,0.848643,1.135118,0.956876


## 레이어 조합

In [62]:
df2 = pd.read_csv('/Users/hanjun/Desktop/python/비타민/추천시스템/1117img_layer2_feature.csv')
df2 = df2.iloc[:,1:]
df3 = pd.read_csv('/Users/hanjun/Desktop/python/비타민/추천시스템/1117img_layer3_feature.csv')
df3 = df3.iloc[:,1:]
df4 = pd.read_csv('/Users/hanjun/Desktop/python/비타민/추천시스템/1117img_layer4_feature.csv')
df4 = df4.iloc[:,1:]

In [63]:
df2.shape

(1117, 129)

In [67]:
for i in range(0,df2.shape[1]):
    df2.rename(columns={f'{i}':f'L2_{i}'}, inplace=True)

In [68]:
for i in range(0,df3.shape[1]):
    df3.rename(columns={f'{i}':f'L3_{i}'}, inplace=True)

In [69]:
for i in range(0,df4.shape[1]):
    df4.rename(columns={f'{i}':f'L4_{i}'}, inplace=True)

In [74]:
df4.to_csv('/Users/hanjun/Desktop/python/비타민/추천시스템/1117img_layer4_feature.csv')

In [82]:
df22 = df2.iloc[:,:]
df33 = df3.iloc[:,1:]

In [84]:
df33.head()

,L3_0,L3_1,L3_2,L3_3,L3_4,L3_5,L3_6,L3_7,L3_8,L3_9,...,L3_246,L3_247,L3_248,L3_249,L3_250,L3_251,L3_252,L3_253,L3_254,L3_255
0,0.143560,0.125398,0.153696,0.086413,0.111541,0.152632,0.086530,0.133514,0.141122,0.153938,...,0.073897,0.097345,0.070567,0.101202,0.104107,0.115527,0.175187,0.119488,0.171178,0.080127
1,0.150055,0.132560,0.161011,0.093052,0.088966,0.151671,0.090369,0.137598,0.158817,0.155494,...,0.078018,0.101547,0.072300,0.096339,0.094771,0.109335,0.164068,0.118924,0.179331,0.090056
2,0.157220,0.132151,0.153932,0.071520,0.098532,0.157952,0.080284,0.125578,0.139754,0.152090,...,0.065017,0.088855,0.061984,0.099535,0.071978,0.103300,0.149619,0.119341,0.173510,0.083480
3,0.144177,0.114252,0.162136,0.084627,0.094893,0.156761,0.090104,0.124675,0.145096,0.157779,...,0.074423,0.105912,0.073771,0.098966,0.106895,0.107317,0.173850,0.081925,0.173054,0.071451
4,0.141616,0.127117,0.159283,0.082114,0.101515,0.156281,0.089998,0.123392,0.148995,0.169307,...,0.052647,0.090081,0.070334,0.096456,0.097238,0.108363,0.170535,0.113151,0.162212,0.064901


In [85]:
df23 = pd.concat([df22,df33], axis=1)

In [86]:
df23.shape

(1117, 385)

In [87]:
df23.head()

,img_name,L2_0,L2_1,L2_2,L2_3,L2_4,L2_5,L2_6,L2_7,L2_8,...,L3_246,L3_247,L3_248,L3_249,L3_250,L3_251,L3_252,L3_253,L3_254,L3_255
0,gyeongnam_203.jpg,0.139648,0.169798,0.136722,0.108873,0.176598,0.123397,0.120343,0.245084,0.149954,...,0.073897,0.097345,0.070567,0.101202,0.104107,0.115527,0.175187,0.119488,0.171178,0.080127
1,gyeongbuk_194.jpg,0.140625,0.193402,0.150842,0.108315,0.186995,0.132722,0.113584,0.244671,0.152429,...,0.078018,0.101547,0.072300,0.096339,0.094771,0.109335,0.164068,0.118924,0.179331,0.090056
2,gyeongnam_217.jpg,0.141954,0.178325,0.142441,0.102194,0.190987,0.121189,0.114551,0.233060,0.150725,...,0.065017,0.088855,0.061984,0.099535,0.071978,0.103300,0.149619,0.119341,0.173510,0.083480
3,jeju_102.jpg,0.151979,0.173062,0.145052,0.106857,0.177310,0.133428,0.102689,0.248616,0.144603,...,0.074423,0.105912,0.073771,0.098966,0.106895,0.107317,0.173850,0.081925,0.173054,0.071451
4,jeju_116.jpg,0.132614,0.175584,0.139151,0.111856,0.175659,0.150412,0.101449,0.255162,0.157335,...,0.052647,0.090081,0.070334,0.096456,0.097238,0.108363,0.170535,0.113151,0.162212,0.064901


In [88]:
df23.columns

Index(['img_name', 'L2_0', 'L2_1', 'L2_2', 'L2_3', 'L2_4', 'L2_5', 'L2_6',
       'L2_7', 'L2_8',
       ...
       'L3_246', 'L3_247', 'L3_248', 'L3_249', 'L3_250', 'L3_251', 'L3_252',
       'L3_253', 'L3_254', 'L3_255'],
      dtype='object', length=385)

In [89]:
df23.to_csv('/Users/hanjun/Desktop/python/비타민/추천시스템/1117img_layer2+3_feature.csv')

In [90]:
df33 = df3.iloc[:,:]
df44 = df4.iloc[:,1:]

df34 = pd.concat([df33,df44], axis=1)

In [91]:
df34.shape

(1117, 769)

In [92]:
df34.columns

Index(['img_name', 'L3_0', 'L3_1', 'L3_2', 'L3_3', 'L3_4', 'L3_5', 'L3_6',
       'L3_7', 'L3_8',
       ...
       'L4_502', 'L4_503', 'L4_504', 'L4_505', 'L4_506', 'L4_507', 'L4_508',
       'L4_509', 'L4_510', 'L4_511'],
      dtype='object', length=769)

In [93]:
df34.to_csv('/Users/hanjun/Desktop/python/비타민/추천시스템/1117img_layer3+4_feature.csv')

In [94]:
df22 = df2.iloc[:,:]
df44 = df4.iloc[:,1:]

df24 = pd.concat([df22,df44], axis=1)

In [95]:
df24.shape

(1117, 641)

In [96]:
df24.columns

Index(['img_name', 'L2_0', 'L2_1', 'L2_2', 'L2_3', 'L2_4', 'L2_5', 'L2_6',
       'L2_7', 'L2_8',
       ...
       'L4_502', 'L4_503', 'L4_504', 'L4_505', 'L4_506', 'L4_507', 'L4_508',
       'L4_509', 'L4_510', 'L4_511'],
      dtype='object', length=641)

In [97]:
df24.to_csv('/Users/hanjun/Desktop/python/비타민/추천시스템/1117img_layer2+4_feature.csv')

In [99]:
df22 = df2.iloc[:,:]
df33 = df3.iloc[:,1:]
df44 = df4.iloc[:,1:]

df234 = pd.concat([df22,df33,df44], axis=1)

In [101]:
df234.shape

(1117, 897)

In [102]:
df234.columns

Index(['img_name', 'L2_0', 'L2_1', 'L2_2', 'L2_3', 'L2_4', 'L2_5', 'L2_6',
       'L2_7', 'L2_8',
       ...
       'L4_502', 'L4_503', 'L4_504', 'L4_505', 'L4_506', 'L4_507', 'L4_508',
       'L4_509', 'L4_510', 'L4_511'],
      dtype='object', length=897)

In [103]:
df234.to_csv('/Users/hanjun/Desktop/python/비타민/추천시스템/1117img_layer2+3+4_feature.csv')